# Organoid Morphological Analysis


## Imports

In [1]:
# This sets the matplotlib backend to be interactive
%matplotlib widget

In [ ]:
# Import packages and helper functions from code folder
from helper_functions import *
import datetime as dt
import importlib
import sys
import psutil
from dask.distributed import Client
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import umap
import pandas as pd
from sklearn.manifold import TSNE



## If you wanna use dask distributed, execute the cell below

In [ ]:
def calculate_dask_parameters(percentage=100):
    # Ensure the percentage is between 1 and 100
    percentage = max(1, min(percentage, 100))
    
    # Calculate the fraction of resources to use
    fraction = percentage / 100.0
    
    # Get the number of physical cores and logical processors
    physical_cores = psutil.cpu_count(logical=False)
    logical_processors = psutil.cpu_count(logical=True)
    
    # Get the total memory in bytes and convert it to GB
    total_memory_gb = psutil.virtual_memory().total / (1024 ** 3)
    
    # Calculate the number of workers and threads per worker to use
    n_workers = max(1, int(physical_cores * fraction))
    n_threads_per_worker = max(1, int((logical_processors * fraction) // n_workers))
    
    # Calculate the memory limit per worker
    memory_per_worker = (total_memory_gb * fraction) / n_workers
    
    return {
        "n_workers": n_workers,
        "threads_per_worker": n_threads_per_worker,
        "memory_limit": f"{memory_per_worker:.2f}GB"
    }

def create_dask_client(percentage=100):
    params = calculate_dask_parameters(percentage)
    client = Client(n_workers=params["n_workers"],
                    threads_per_worker=params["threads_per_worker"],
                    memory_limit=params["memory_limit"])
    return client

# Usage
percentage_to_use = 90  # Example: use 75% of the computer's resources
client = create_dask_client(percentage_to_use)


## Data Declaration

In [ ]:
# Specify the image folder path and output json path
folder_path = "/files/data"
output_folder = "/files/data/results"
output_prefix = "B22_D34_H9_QPRT_KO"
target = "condition"

# Create the output folder if it does not exist
os.makedirs(output_folder, exist_ok=True)

outputname = dt.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")+"_"+output_prefix
output_json_path = output_folder+"/"+outputname+"metadata.json"

## Reads Metadata from Filenames 
To do metadata read from input file provided by user

In [ ]:
# Specifiy the extra metadata and condition rules
extra_metadata = {
}

# Conditional rules included in the metadata
condition_rules = {
    'day': {
        'd17' : "Day 17",
        'd24' : "Day 24",
        'd34' : "Day 34",
        'd36' : "Day 36",
        'd45' : "Day 45",
        'd47' : "Day 47",
        'd49' : "Day 49",
    },
    'condition' : {
        'WT' : "WT",
        'NTC' : "NTC",
        'Q2.1' : "Q2.1",
        'Q2.2' : "Q2.2",
    },
    'molar' : {
        '0mM' : "0mM PA",
        '5mM' : "5mM PA",
        'HIL6' : "HIL6",
        'default' : "No PA"
    }, 
    'excluded' : {
        'Trash' : True,
        'TRASH' : True
    }
}

image_metadata = get_image_metadata(folder_path, extra_metadata, condition_rules)
save_metadata_as_xlsx(image_metadata, output_folder+"/"+outputname+"_image_metadata.xlsx")

image_files = list(image_metadata.keys())

# Filter image_metadata where 'molar' is '0mM PA'
filtered_metadata = {k: v for k, v in image_metadata.items() if v['excluded'] != True}
filtered_image_files = list(filtered_metadata.keys())


print(f"Number of images: {len(image_files)}")
print(f"Number of images after filtering: {len(filtered_image_files)}")
display(pd.DataFrame(filtered_metadata).T)

image_metadata = filtered_metadata
image_files = filtered_image_files

## Run analysis and update metadata with the values

for 75 images the analysis takes about 1m 44 sec (i.e. ~1.5sec per image)

In [ ]:
metadata, statistics, segmented_organoids, hu_moments, flags, curvatures, inflection_points, smooth_contours = update_metadata_with_analysis(folder_path, image_metadata, image_files)


## plots the flagged images into a pdf file

In [ ]:

# Create a PDF file to save the plots

if(flags):
    pdf_path = output_folder+"/"+outputname+"_flagged_images.pdf"
    with PdfPages(pdf_path) as pdf:
        # Iterate through the flagged images
        for i, image_file in enumerate(image_files):
            if flags[i]:  # Check if there are any flags for the current image
                image = segmented_organoids[i]  # Get the corresponding organoid image
                flag_text = ', '.join(flags[i])  # Join the flags into a single string

                # Plot the image
                plt.figure()
                plt.imshow(image, cmap='gray')
                plt.title(image_file)
                plt.xlabel(flag_text)
                pdf.savefig() # Save the current figure to the PDF
                plt.close()  # Close the figure to free up memory

    print(f"PDF saved to {pdf_path}")
else:
    print("No flagged images found")


## Execute this cell if you want to remove all the flagged organoids from the metadata

In [ ]:
metadata = remove_flagged_from_metadata(metadata, flags)

print(len(segmented_organoids))
# Create an index of images not in metadata.keys
index_images = [True if image in metadata.keys() else False for image in image_files] 
statistics = [data for data, include in zip(statistics, index_images) if include]   
segmented_organoids =  [data for data, include in zip(segmented_organoids, index_images) if include]
hu_moments= [data for data, include in zip(hu_moments, index_images) if include]
curvatures = [data for data, include in zip(curvatures, index_images) if include]
inflection_points = [data for data, include in zip(inflection_points, index_images) if include]
smooth_contours = [data for data, include in zip(smooth_contours, index_images) if include]
print(len(segmented_organoids))



## Execute this cell to save the metadata to an Excel sheet

In [ ]:
save_metadata_as_xlsx(metadata, output_folder+"/"+outputname+"_analysis_metadata.xlsx")

## Execute this cell to read the metadata from the provided Excel sheet

In [ ]:
#save_metadata_as_xlsx(metadata, "analysis_metadata.xlsx")
# metadata = read_metadata_from_xlsx("analysis_metadata.xlsx")

## Converts metadata to dataframe

In [ ]:
# Convert metadata to DataFrame
df = pd.DataFrame.from_dict(metadata, orient='index').drop(columns=['background_uniformity'])

# Convert the 'hu_moments' column from string to list of numbers
#df['hu_moments'] = df['hu_moments'].apply(ast.literal_eval)

# Expand hu_moments into separate columns
hu_moments_df = pd.DataFrame(df['hu_moments'].tolist(), index=df.index)
hu_moments_df.columns = [f'hu_moment_{i}' for i in range(1, hu_moments_df.shape[1] + 1)]

# Concatenate the original dataframe with the new hu_moments columns
df = pd.concat([df, hu_moments_df], axis=1)

# Drop the original hu_moments column if it's no longer needed
df.drop(columns=['hu_moments'], inplace=True)

### create a pdf with umap plot

In [ ]:
# Import StandardScaler
# Select features for UMAP
feature_columns = ['circularity', 'roundness', 'area', 'perimeter', 'feret max', 'feret min', 'contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation', 'aspect_ratio', 'dne', 'inflection_points', 'background_transparency'] + hu_moments_df.columns.tolist()


X = df[feature_columns].values

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Perform UMAP dimensionality reduction
umap_model = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=2, random_state=42)
umap_embedding = umap_model.fit_transform(X_scaled)

# Add UMAP results to DataFrame
df['umap_x'] = umap_embedding[:, 0]
df['umap_y'] = umap_embedding[:, 1]

# Perform T-SNE dimensionality reduction
tsne_model = TSNE(n_components=2, random_state=42)
tsne_embedding = tsne_model.fit_transform(X_scaled)

# Add T-SNE results to DataFrame
df['tsne_x'] = tsne_embedding[:, 0]
df['tsne_y'] = tsne_embedding[:, 1]

pdf_path = output_folder+"/"+outputname+"_UMAP.pdf"
with PdfPages(pdf_path) as pdf:
    scatter = plt.scatter(df['umap_x'], df['umap_y'], 
                          c=df[target].astype('category').cat.codes, 
                          cmap='viridis', s=5)
    # Fix legend
    handles, _ = scatter.legend_elements()
    plt.legend(handles, df[target].astype('category').cat.categories, 
               title=target, 
               loc='upper center', ncol = len(np.unique_values(df[target])),
               bbox_to_anchor=(0.5, 1.15))
    pdf.savefig()
    plt.close()
    print(f"PDF saved to {pdf_path}")

# Save the DataFrame with UMAP and T-SNE results
save_df_as_xlsx(df, output_folder+"/"+outputname+  "_analysis_metadata_all_cleaned.xlsx")

display(df)

In [ ]:
%run /files/dashapp_current.py

# Plotting

In [ ]:
# Step 3: Visualize the clusters with images as markers

plot_with_images(df, target, segmented_organoids, output_folder, outputname)


## Plot the Analysis with the contour coloured in the DNE

In [ ]:
for i in tqdm(range(5)):
    plot_analysis(segmented_organoids[i], statistics[i], flags[i], output_folder=output_folder,output_name=f'analysis_{image_files[i].split(".")[0]}')
    plot_contour_with_dne(segmented_organoids[i], smooth_contours[i], curvatures[i], inflection_points[i], output_folder=output_folder,output_name=f'analysis_{image_files[i].split(".")[0]}')

## Decision trees and Random Forest Classifier

### Imports for Decision Trees and RF

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import mplcursors
from skimage import data, segmentation, feature, future
from sklearn.ensemble import RandomForestClassifier
from functools import partial

### Load Image

In [ ]:
image = cv2.imread(os.path.join(folder_path, image_files[27]))


#image = segmented_organoids[0]
image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

plt.figure()
plt.imshow(image)
plt.show()

In [ ]:


# Load an example image
img = image

# Initialize training labels
training_labels = np.zeros(img.shape[:2], dtype=np.uint8)

# Display the image
fig, ax = plt.subplots()
ax.imshow(img)
plt.title("Click to label regions. Press '1'-'5' to change label.")
cursor = mplcursors.cursor(hover=True)

# Keep track of current label
current_label = [1]  # Use a list to allow modification within the event handler

def update_label(event):
    if event.key in '12345':
        current_label[0] = int(event.key)
        print(f"Current label: {current_label[0]}")

def on_click(event):
    if event.inaxes:
        x, y = int(event.xdata), int(event.ydata)
        radius = 10  # Adjust the radius as needed
        training_labels[max(y-radius, 0):min(y+radius, img.shape[0]), max(x-radius, 0):min(x+radius, img.shape[1])] = current_label[0]
        ax.add_patch(plt.Circle((x, y), radius, color='red', fill=False))
        fig.canvas.draw()

fig.canvas.mpl_connect('button_press_event', on_click)
fig.canvas.mpl_connect('key_press_event', update_label)
plt.show()




In [ ]:
# After labeling, proceed with segmentation
sigma_min = 1
sigma_max = 32
features_func = partial(
    feature.multiscale_basic_features,
    intensity=True,
    edges=True,
    texture=True,
    sigma_min=sigma_min,
    sigma_max=sigma_max,
    channel_axis=-1
)
features = features_func(img)

clf = RandomForestClassifier(n_estimators=50, n_jobs=-1, max_depth=10, max_samples=0.05)
clf = future.fit_segmenter(training_labels, features, clf)
result = future.predict_segmenter(features, clf)

# Plot the results
fig, ax = plt.subplots(1, 2, sharex=True, sharey=True, figsize=(9, 4))
ax[0].imshow(segmentation.mark_boundaries(img, result, mode='thick'))
ax[0].contour(training_labels, colors='red', linewidths=1)
ax[0].set_title('Image, mask and segmentation boundaries')
ax[1].imshow(result)
ax[1].set_title('Segmentation')
fig.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(9, 4))
l = len(clf.feature_importances_)
feature_importance = (
    clf.feature_importances_[: l // 3],
    clf.feature_importances_[l // 3 : 2 * l // 3],
    clf.feature_importances_[2 * l // 3 :],
)
sigmas = np.logspace(
    np.log2(sigma_min),
    np.log2(sigma_max),
    num=int(np.log2(sigma_max) - np.log2(sigma_min) + 1),
    base=2,
    endpoint=True,
)
for ch, color in zip(range(3), ['r', 'g', 'b']):
    ax[0].plot(sigmas, feature_importance[ch][::3], 'o', color=color)
    ax[0].set_title("Intensity features")
    ax[0].set_xlabel("$\\sigma$")
for ch, color in zip(range(3), ['r', 'g', 'b']):
    ax[1].plot(sigmas, feature_importance[ch][1::3], 'o', color=color)
    ax[1].plot(sigmas, feature_importance[ch][2::3], 's', color=color)
    ax[1].set_title("Texture features")
    ax[1].set_xlabel("$\\sigma$")

fig.tight_layout()

In [ ]:
img_path = "/Users/andreas_chiocchetti/develop/Organoid_Morphology/data/OneDrive_75_12-10-2024/20240229_DH_2x_B22_H9_NTC_1_dish2_6.bmp"

In [ ]:
# Save the CLF as a pickle file
import pickle
with open('clf.pkl', 'wb') as f:
    pickle.dump(clf, f)



In [ ]:
segmentations = []
for i in tqdm(range(len(image_files)), desc = 'Segmenting images'):
    image = cv2.imread(os.path.join(image_path, image_files[i]))
    # image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    img_new = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)

    features_new = features_func(img_new)
    segmentations.append(future.predict_segmenter(features_new, clf))

In [ ]:
#image = cv2.imread(os.path.join(image_path, image_files[2]))
## image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
#image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
#img_new = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)


for image in segmented_organoids:

    img_new = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)


    features_new = features_func(img_new)
    result_new = future.predict_segmenter(features_new, clf)
    fig, ax = plt.subplots(1, 2, sharex=True, sharey=True, figsize=(6, 4))
    ax[0].imshow(segmentation.mark_boundaries(img_new, result_new, mode='thick'))
    ax[0].set_title('Image')
    ax[1].imshow(result_new)
    cbar = plt.colorbar(ax[1].imshow(result_new), ax=ax[1], orientation='vertical')
    ax[1].set_title('Segmentation')
    fig.tight_layout()

    plt.show()

In [ ]:
for i in tqdm(range(len(image_files)), desc = 'Plotting images'):
    image = cv2.imread(os.path.join(image_path, image_files[i]))

    fig, ax = plt.subplots(1, 2, sharex=True, sharey=True, figsize=(6, 4))
    ax[0].imshow(segmentation.mark_boundaries(image, segmentations[i], mode='thick'))
    ax[0].set_title('Image')
    ax[1].imshow(segmentations[i])
    ax[1].set_title('Segmentation')
    fig.tight_layout()

    plt.show()